In [ ]:
#==========================================
# Title:  Climate Credit Risk Model
# Author: Vachan
# Email : vachan@iitb.ac.in
#==========================================

In [9]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from itertools import combinations

In [10]:
def simulate_future_data(base_data, best_model_vars, years_to_simulate=4):
    """
    Simulates future data for each customer_id based on logical trends.

    Args:
        base_data (pd.DataFrame): The original dataframe to base simulations on. 
                                  Must contain a 'customer_id' column.
        best_model_vars (list): The list of variables to simulate.
        years_to_simulate (int): The number of future years to simulate.

    Returns:
        pd.DataFrame: A dataframe containing the original and simulated data.
    """
    print("--- Starting Future Data Simulation (per customer) ---")
    
    if 'customer_id' not in base_data.columns:
        raise ValueError("The base data must contain a 'customer_id' column.")

    # Start with the 2021 data.
    last_known_data = base_data.copy()
    if 'year' not in last_known_data.columns:
        last_known_data['year'] = 2021
    
    # Ensure credit_history is numeric for simulation
    if 'credit_history' in best_model_vars:
        last_known_data['credit_history'] = pd.to_numeric(last_known_data['credit_history'], errors='coerce').fillna(0)

    future_data_list = [last_known_data] # Include the base year in the list

    for year_offset in range(1, years_to_simulate + 1):
        future_year = 2021 + year_offset
        print(f"Simulating data for year: {future_year}")
        
        future_df = last_known_data.copy()
        future_df['year'] = future_year

        # --- Simulation Logic (applied to the previous year's data) ---
        if 'ltv' in best_model_vars:
            future_df['ltv'] *= np.random.uniform(0.95, 0.99, size=len(future_df))

        if 'average_age' in best_model_vars:
            future_df['average_age'] += 1

        if 'credit_score' in best_model_vars:
            future_df['credit_score'] *= np.random.uniform(1.0, 1.02, size=len(future_df))
            future_df['credit_score'] = future_df['credit_score'].clip(upper=900)

        if 'last_six_month_defaulted_no' in best_model_vars:
            future_df['last_six_month_defaulted_no'] = np.random.choice([0, 1], size=len(future_df), p=[0.98, 0.02])
            
        if 'credit_history' in best_model_vars:
            future_df['credit_history'] += 1

        future_data_list.append(future_df)
        last_known_data = future_df.copy()
    
    full_simulated_df = pd.concat(future_data_list, ignore_index=True)
    
    print("\n--- Simulation Complete ---")
    return full_simulated_df

In [11]:
def load_ngfs_data_for_country(file_path, country_name, ngfs_variable_vector, years_to_extract):
    """
    Loads and filters NGFS data from an Excel file for a specific country, 
    handling different scenarios correctly.

    Args:
        file_path (str): Path to the NGFS-structured Excel file.
        country_name (str): The name of the country to filter for (e.g., 'India - IND').
        ngfs_variable_vector (list): A list of the 'Variable' names to extract.
        years_to_extract (list): A list of integer years to extract.

    Returns:
        pd.DataFrame: A cleaned dataframe with year and Scenario as rows and variables as columns.
    """
    print(f"--- Loading NGFS data from {file_path} for {country_name} ---")
    
    try:
        df = pd.read_excel(file_path)
        df.columns = [str(col).strip() for col in df.columns]
        
        required_cols = ['Region', 'Scenario', 'Variable']
        if not all(col in df.columns for col in required_cols):
            raise ValueError(f"The Excel file must contain {required_cols} columns.")

    except Exception as e:
        print(f"Error loading or parsing the Excel file: {e}")
        return pd.DataFrame()

    # Filter for the specified country and variables
    country_df = df[df['Region'] == country_name]
    filtered_df = country_df[country_df['Variable'].isin(ngfs_variable_vector)]

    if filtered_df.empty:
        print(f"Warning: No data found for country '{country_name}' or specified variables.")
        return pd.DataFrame()

    # Melt the dataframe to turn year columns into rows
    id_vars = ['Region', 'Scenario', 'Variable']
    year_cols = [str(y) for y in years_to_extract]
    value_vars = [col for col in year_cols if col in filtered_df.columns]
    
    if not value_vars:
        print(f"Warning: No data found for the years {years_to_extract}.")
        return pd.DataFrame()

    melted_df = filtered_df.melt(
        id_vars=id_vars, 
        value_vars=value_vars, 
        var_name='year', 
        value_name='value'
    )

    # Pivot the table to get variables as columns
    pivoted_df = melted_df.pivot_table(
        index=['year', 'Scenario'], 
        columns='Variable', 
        values='value'
    ).reset_index()

    pivoted_df['year'] = pd.to_numeric(pivoted_df['year']).astype(int)
    
    print("NGFS data loaded and transformed successfully.")
    return pivoted_df


In [12]:
def find_best_regression_models(data, dependent_vars, explanatory_vars):
    """
    For each scenario and dependent variable, finds the best MULTIPLE linear 
    regression model by iterating through all combinations of explanatory variables
    and selecting the one with the lowest AIC.
    """
    print("\n\n--- Starting Multiple Regression Model Search (per Scenario) ---")
    
    if 'Scenario' not in data.columns:
        print("Warning: 'Scenario' column not found. Running a single set of models.")
        scenarios = ['Default']
        data['Scenario'] = 'Default'
    else:
        scenarios = data['Scenario'].unique()

    for scenario in scenarios:
        print(f"\n==================== SCENARIO: {scenario} ====================")
        scenario_data = data[data['Scenario'] == scenario]

        for dep_var in dependent_vars:
            lowest_aic = float('inf')
            best_model_summary = None
            best_combination = None
            
            print(f"\n--- Finding best model for: {dep_var} ---")

            # Iterate through all possible numbers of predictors (from 1 to all)
            for i in range(1, len(explanatory_vars) + 1):
                # Iterate through all combinations of that size
                for combo in combinations(explanatory_vars, i):
                    combo_list = list(combo)
                    
                    if dep_var in combo_list:
                        continue

                    model_data = scenario_data[[dep_var] + combo_list].dropna()
                    
                    # Ensure there are enough data points to fit the model
                    if len(model_data) < len(combo_list) + 2:
                        continue

                    X = model_data[combo_list]
                    y = model_data[dep_var]
                    
                    X = sm.add_constant(X)

                    try:
                        model = sm.OLS(y, X).fit()
                        current_aic = model.aic

                        if current_aic < lowest_aic:
                            lowest_aic = current_aic
                            best_model_summary = model.summary()
                            best_combination = combo_list
                    except Exception:
                        # Pass silently on model fitting errors for invalid combinations
                        pass
            
            if best_model_summary:
                print(f"\n--- Best Model Found for '{dep_var}' in '{scenario}' Scenario ---")
                print(f"The best combination of explanatory variables is: {best_combination}")
                print(f"Lowest AIC: {lowest_aic:.4f}")
                print("\n--- Best Model Summary ---")
                print(best_model_summary)
            else:
                print(f"\nCould not find a suitable model for '{dep_var}' in '{scenario}'.")


In [13]:
# --- Main Execution ---
if __name__ == '__main__':
    try:
        # --- Define File Paths and Variables ---
        customer_data_file = "C:/Users/vacha/OneDrive - Indian Institute of Technology Bombay/Climate Finance/Retail Portfoilio Research/Auto Risk Kaggle/data.csv"
        ngfs_data_file = "C:/Users/vacha/OneDrive - Indian Institute of Technology Bombay/Climate Finance/Retail Portfoilio Research/GEME3_IIASA_2025_05_02.xlsx" # <--- CHANGE THIS PATH

        ngfs_variables_to_use = [
            'Imports|Crude Oil',
            'Crude Oil YoY Growth',
            'Expenditure|Household|Transport services',
            'Investment|Energy',
            'Exports|EV Transport Equipment',
            'Production|EV Transport Equipment',
            'Investments|EV Transport Equipment',
            'Expenditure|Household|Purchase of vehicles',
            'Power Generation Technologies|Oil fired',
            'Unemployment Rate'
        ]

        best_model_variables = [
            'ltv', 'average_age', 'credit_score', 
            'last_six_month_defaulted_no', 'credit_history'
        ]
        
        # --- 1. Load Data ---
        df = pd.read_csv(customer_data_file)
        
        explanatory_df = load_ngfs_data_for_country(
            file_path=ngfs_data_file,
            country_name='India - IND',
            ngfs_variable_vector=ngfs_variables_to_use,
            years_to_extract=[2021, 2022, 2023, 2024, 2025]
        )
        
        if explanatory_df.empty:
            raise ValueError("Failed to load explanatory data.")

        df.dropna(subset=best_model_variables, inplace=True)

        # --- 2. Simulate Future Customer-Level Data ---
        customer_data_simulated = simulate_future_data(df, best_model_variables)
        
        # --- 3. Merge Simulated Data with Explanatory Data ---
        # This merge expands the customer data to include rows for each scenario.
        final_data = pd.merge(customer_data_simulated, explanatory_df, on='year', how='left')
        final_data.dropna(subset=explanatory_df.columns.drop('year'), inplace=True)

        print("\n--- Merged Data Sample ---")
        print(final_data.sample(5))
        
        # --- 4. Find Best Regression Models ---
        explanatory_variable_vector = [col for col in explanatory_df.columns if col not in ['year', 'Scenario']]

        find_best_regression_models(
            final_data, 
            best_model_variables, 
            explanatory_variable_vector
        )

    except FileNotFoundError as e:
        print(f"\nError: A required file was not found. Details: {e}")
    except Exception as e:
        print(f"An error occurred: {e}")


--- Loading NGFS data from C:/Users/vacha/OneDrive - Indian Institute of Technology Bombay/Climate Finance/Retail Portfoilio Research/GEME3_IIASA_2025_05_02.xlsx for India - IND ---
NGFS data loaded and transformed successfully.
--- Starting Future Data Simulation (per customer) ---
Simulating data for year: 2022
Simulating data for year: 2023
Simulating data for year: 2024
Simulating data for year: 2025

--- Simulation Complete ---

--- Merged Data Sample ---
          customer_id  main_account_loan_no  main_account_active_loan_no  \
12451101      4703625                    13                            2   
2422553       5117431                     0                            0   
6931508       6495011                     2                            2   
2386170       4360946                     0                            0   
9018611       4747851                     0                            0   

          main_account_overdue_no  main_account_outstanding_loan  \
12451101  